In [4]:
from loguru import logger
from datetime import timedelta
import pandas as pd
import json
import requests
import numpy as np

import plotly.express as px
import datetime as dt

import math
import time

import haversine

%reload_ext autoreload
%autoreload 2

import plotly.graph_objects as go

import sys
sys.path.append('..')
from src.co2_modells import co2_modell, co2_modell_direct
from src.data_matrix import create_dict_points, create_df_distance_matrix
from src.data_for_tool import data_for_tool, create_table_daily_mean, create_table_total
from src.data_for_tool import create_lists_days_weeks_months, create_list_days_month, remove_nan_distance_matrix

truck_capacity = 13000

In [5]:
def get_results_volume_variation(truck_capacity = truck_capacity, mode = "combined", algorithm = "base"):
    # daily
    t = time.process_time()
    df_results = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad"])
    for volume in [1,5,10]:
        for i in range(1,len(list_days[:8])):
            try: 
                dict_results, df_results_details = co2_modell(
                df_dc1, df_distance_matrix, dict_terminals, list_days[i-1], list_days[i], 
                volume = volume, mode = mode, truck_capacity = truck_capacity)
                dict_results["volume"] = volume
                df_results = df_results.append(dict_results, ignore_index=True)
            except:
                pass
    return df_results

In [6]:
def get_results_one_month(df, month, volume, truck_capacity = truck_capacity, mode = "combined", algorithm = "base"):
    list_days = create_list_days_month(2022, month)
    # daily
    t = time.process_time()
    df_results = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad"])
    for i in range(1,len(list_days)):
        try: 
            dict_results, df_results_details = co2_modell(
            df, df_distance_matrix, dict_terminals, list_days[i-1], list_days[i], 
            volume = volume, mode = mode, truck_capacity = truck_capacity)
            dict_results["volume"] = volume
            df_results = df_results.append(dict_results, ignore_index=True)
        except:
            pass
    df_results.to_csv(f"results/results_daily_{mode}_{algorithm}_0{month}_v{volume}.csv", index = False)
    return df_results

In [7]:
def get_results(df, list_days, planning_horizon, volume, truck_capacity = truck_capacity, mode = "combined", algorithm = "base"):
    t = time.process_time()
    file_path = f"results/results_{planning_horizon}_v{volume}_{mode}_{algorithm}.csv"
    df_results = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad"])
    for i in range(0,len(list_days)-1):
        logger.info(f"From {list_days[i]} to {list_days[i+1]}")
        try: 
            dict_results, df_results_details = co2_modell(
                df, df_distance_matrix, dict_terminals, list_days[i], list_days[i+1], 
                volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 40)
        except:
            try:
                dict_results, df_results_details = co2_modell(
                    df, df_distance_matrix, dict_terminals, list_days[i], list_days[i+1], 
                    volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 60)
            except:
                try:
                    dict_results, df_results_details = co2_modell(
                        df, df_distance_matrix, dict_terminals, list_days[i], list_days[i+1], 
                        volume = volume, mode = mode, truck_capacity = truck_capacity, nb_trucks = 80)
                except:
                    logger.info("No results")
        dict_results["volume"] = volume
        df_results = df_results.append(dict_results, ignore_index=True)
        df_results.to_csv(f"results/results_{planning_horizon}_v{volume}_{mode}_{algorithm}.csv", index = False)
    return df_results

# Data Import

In [8]:
# Import data
project_path = 'results'
dataset_name = 'df_limited'
df = pd.read_csv(f'{project_path}/{dataset_name}.csv', parse_dates=['Delivery date'])
dataset_name = 'df_distance_matrix'
df_distance_matrix = pd.read_csv(f'{project_path}/{dataset_name}.csv', index_col = 0)

# Import dict terminals
f = open("../data/external/intermodal_terminals/dict_terminals.json")
dict_terminals = json.load(f)

# Import dict points
f = open("../data/processed/dict_points.json")
dict_points = json.load(f)

#df_dc2, df_dc3, df_distance_matrix, dict_terminals, df_terminal = data_for_tool()
list_days, list_mondays, list_bi_daily, list_months = create_lists_days_weeks_months(2022)
list_mondays.append(dt.date(2023, 1, 2))

df_dc1 = df[df["Shipper name"]=="DC1"]
df_dc2 = df[df["Shipper name"]=="DC2"]

len(df["Receiver name"].unique())

787

In [9]:
remove_nan_distance_matrix(df, df_distance_matrix)

[]


(           DC1       DC2      C129      C122      C133      C109      C138  \
 DC1        0.0  583763.2   86015.7  463258.7  615313.4   86445.3  561141.1   
 DC2   581011.6       0.0  541660.6  440804.3  462825.1  508467.9   24726.3   
 C129   85468.8  550743.5       0.0  478484.6  608632.8  103209.2  528121.4   
 C122  444948.7  442362.8  478236.8       0.0  159067.4  362540.1  419740.8   
 C133  615272.6  465374.2  607492.0  158892.4       0.0  532864.0  454953.8   
 ...        ...       ...       ...       ...       ...       ...       ...   
 T6    564288.1   22383.7  524937.3  424080.9  463656.3  491744.5   11227.5   
 T7    590887.8  501278.9  583107.2  171814.4   34545.8  508479.2  490858.5   
 T8    896265.7  485482.1  819532.6  878154.2  930430.5  878857.9  472843.1   
 T9    284953.8  700403.8  294629.1  406047.5  558102.2  200615.9  677781.8   
 T10   401887.3  216263.3  362536.4  228316.1  303256.5  329343.7  193641.2   
 
           C140      C151        C38  ...        T

In [10]:
len(df["Receiver name"].unique())

787

***

In [8]:
df_results = pd.DataFrame(columns = ["date from", "date to", "co2 road", "distance road", "routes road", "processing time road","co2 railroad", "distance railroad", "terminal allocation", "routes railroad", "processing time railroad", "volume"])

In [14]:
df_results.to_csv(f"results/results_{planning_horizon}_v{volume}_{mode}_{algorithm}.csv", index = False)

NameError: name 'planning_horizon' is not defined

In [18]:
mode = "combined"
algorithm = "base"
volume = 10
month = "01"
df_results = pd.read_csv(f"results/DC2/results_daily_{mode}_{algorithm}_{month}_v{volume}.csv")
for month in ["02","03","04","05","06","07","08","09","10","11","12"]:
    df_results_month = pd.read_csv(f"results/DC2/results_daily_{mode}_{algorithm}_{month}_v{volume}.csv")
    df_results = pd.concat([df_results,df_results_month])

FileNotFoundError: [Errno 2] No such file or directory: 'results/DC2/results_daily_combined_base_10_v10.csv'

In [16]:
import xlsxwriter
mode = "combined"
algorithm = "base"
dict_index_names = {0: "GHG road", 1: "GHG rail", 2:"Difference absolut", 3: "Difference relative"}
writer = pd.ExcelWriter(f'results/results_{mode}_{algorithm}.xlsx', engine='xlsxwriter')
for volume in [1, 5, 10]:
    df_results_daily = pd.read_csv(f"results/results_daily_{mode}_{algorithm}_v{volume}.csv")
    df_results_bidaily = pd.read_csv(f"results/results_bidaily_{mode}_{algorithm}_v{volume}.csv")
    df_results_weekly = pd.read_csv(f"results/results_weekly_{mode}_{algorithm}_v{volume}.csv")
    df_results_daily_mean = pd.concat([
        create_table_daily_mean(df_results_daily, "daily planning"),
        create_table_daily_mean(df_results_bidaily, "bi-daily planning"),
        create_table_daily_mean(df_results_weekly, "weekly planning")], axis = 1)
    df_results_daily_mean = df_results_daily_mean.rename(dict_index_names)
    df_results_total = pd.concat([
        create_table_total(df_results_daily, "daily planning"),
        create_table_total(df_results_bidaily, "bi-daily planning"),
        create_table_total(df_results_weekly, "weekly planning")], axis = 1)
    df_results_total = df_results_total.rename(dict_index_names)
    
    # Write each dataframe to a different worksheet.
    df_results_daily_mean.to_excel(writer, sheet_name=f'Daily mean v{volume}')
    df_results_total.to_excel(writer, sheet_name=f'Total v{volume}')
writer.close()

daily planning :  6.7786604064524685
daily planning :  7.824410548657537
bi-daily planning :  4.928610573425986
bi-daily planning :  5.4191116280077996
weekly planning :  2.253787153128018
weekly planning :  2.203569577754598
daily planning :  8.213949623133754
daily planning :  8.306017353857635
bi-daily planning :  6.53389821149667
bi-daily planning :  5.907647991897648
weekly planning :  4.0892448521044535
weekly planning :  2.6960362653235252
daily planning :  9.961960045446707
daily planning :  8.78944910792969
bi-daily planning :  7.962918428083938
bi-daily planning :  6.442686903995213
weekly planning :  6.710892037942834
weekly planning :  3.6024471046427626


In [66]:
df_results_daily_mean.rename({0: "GHG road", 1: "GHG rail", 2:"Difference absolut", 3: "Difference relative"})

,daily planning,bi-daily planning,weekly planning
GHG road,6778.660406,4928.610573,2253.787153
GHG rail,7824.410549,5419.111628,2203.569578
Difference absolut,-1045.750142,-490.501055,50.217575
Difference relative,-15.427091,-9.952116,2.228142


In [ ]:
df_results_daily_mean = pd.concat([
    create_table_daily_mean(df_results_daily, "daily planning"),
    create_table_daily_mean(df_results_bidaily, "bi-daily planning"),
    create_table_daily_mean(df_results_weekly, "weekly planning")], axis = 1)
df_results_daily_mean

# Application of Decision Support Tool

1. Base scenario: Rail mandatory
2. Scenario: Road allowed
3. Scenario: Relaxation 150/20
4. Scenario: Multi-modal, Combining the two DC's

In [10]:
from src.plots import graph_solution, graph_road, graph_solution_with_radius
from src.create_solution import create_solution, create_solution_150_20

In [11]:
def evaluate_solution(df, date_from, date_to, dict_terminals, list_terminals, volume, truck_capacity = truck_capacity):
    df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
    dict_points = create_dict_points(df, "Shipper name", "Shipper latitude", "Shipper longitude")       
    dict_points.update(create_dict_points(df, "Receiver name", "Receiver latitude", "Receiver longitude"))
    list_solution, closest_dct = create_solution(df, dict_terminals, dict_points)
    list_clients = []
    for i in list_terminals:
        list_clients += list_solution[i-1]
    list_solution, closest_dct = create_solution_150_20(df, dict_terminals, dict_points)
    list_clients += list_solution[int(closest_dct[-1])-1]
    df  = df[df["Receiver name"].isin(list_clients)]
    df = df[df["Sender weight (kg)"]>5]
#     close_clients = df_test[df_test["DC1"]<10000].index.to_list()
#     df = df[~df["Receiver name"].isin(close_clients)]
    dict_results, df_results_details  = co2_modell(
        df, df_distance_matrix, dict_terminals, date_from, date_to, volume = volume)
    return dict_results, df_results_details, closest_dct, df

In [12]:
def evaluate_solution_intermodal_combined(df, date_from, date_to, dict_terminals, list_terminals, volume, truck_capacity = truck_capacity):
    df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
    dict_points = create_dict_points(df, "Shipper name", "Shipper latitude", "Shipper longitude")       
    dict_points.update(create_dict_points(df, "Receiver name", "Receiver latitude", "Receiver longitude"))
    list_solution, closest_dct = create_solution(df, dict_terminals, dict_points)
    list_clients = []
    for i in list_terminals:
        list_clients += list_solution[i-1]
    list_solution, closest_dct = create_solution_150_20(df, dict_terminals, dict_points)
    list_clients += list_solution[int(closest_dct[-1])-1]
    list_clients += ["C316"]
    df  = df[df["Receiver name"].isin(list_clients)]
    df = df[df["Sender weight (kg)"]>5]
#     close_clients = df_test[df_test["DC1"]<10000].index.to_list()
#     df = df[~df["Receiver name"].isin(close_clients)]
    dict_results, df_results_details  = co2_modell(
        df, df_distance_matrix, dict_terminals, date_from, date_to, volume = volume)
    return dict_results, df_results_details, closest_dct, df

In [13]:
def evaluate_solution_intermodal_intermodal(df, date_from, date_to, dict_terminals, list_terminals, volume, truck_capacity = truck_capacity):
    df = df[(df["Delivery date"]>=pd.to_datetime(date_from))&(df["Delivery date"]<=pd.to_datetime(date_to))]
    dict_points = create_dict_points(df, "Shipper name", "Shipper latitude", "Shipper longitude")       
    dict_points.update(create_dict_points(df, "Receiver name", "Receiver latitude", "Receiver longitude"))
    list_solution, closest_dct = create_solution(df, dict_terminals, dict_points)
    list_clients = []
    for i in list_terminals:
        list_clients += list_solution[i-1]
    list_solution, closest_dct = create_solution_150_20(df, dict_terminals, dict_points)
    list_clients += list_solution[int(closest_dct[-1])-1]
    list_clients += ["C316"]
    df  = df[df["Receiver name"].isin(list_clients)]
    df = df[df["Sender weight (kg)"]>5]
#     close_clients = df_test[df_test["DC1"]<10000].index.to_list()
#     df = df[~df["Receiver name"].isin(close_clients)]
    dict_results, df_results_details  = co2_modell(
        df, df_distance_matrix, dict_terminals, date_from, date_to, volume = volume, mode = "intermodal")
    return dict_results, df_results_details, closest_dct, df

## 1. Base scenario

### Daily

In [11]:
#i = 5
df_dc = df_dc2
algorithm = "base"
mode = "combined"

In [19]:
for i in [10]:
    #get_results_one_month(df_dc, i, 1, truck_capacity, mode = mode, algorithm = algorithm)
    #get_results_one_month(df_dc, i, 5, truck_capacity, mode = mode, algorithm = algorithm)
    get_results_one_month(df_dc, i, 10, truck_capacity, mode = mode, algorithm = algorithm)

2023-08-27 15:54:54.529 | INFO     | src.co2_modells:preprocessing_modelling:38 - nan
2023-08-27 15:54:54.533 | INFO     | src.co2_modells:preprocessing_modelling:38 - nan
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-27 15:54:54.535 | INFO     | src.co2_modells:preprocessing_modelling:38 - 1204.96
2023-08-27 15:54:54.540 | INFO     | src.co2_modells:co2_modell:71 - 2.130076923076923


(0, 10)
No deliveries from 2022-10-01 to 2022-10-02
(0, 10)
No deliveries from 2022-10-02 to 2022-10-03
(103, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(16, 12)
(39, 16)


2023-08-27 15:55:05.628 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.628 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.628 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.638 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.666 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.668 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.668 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.668 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:55:05.668 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_24120\3074107163.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: Se

...

Total processing time: 6.046875
(174, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(27, 12)
(59, 16)


2023-08-27 15:58:38.107 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.122 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.128 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.128 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.128 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.138 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.139 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.139 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-27 15:58:38.150 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_24120\3074107163.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-27 15:58:38.155 | INFO     | src.co2_modells:preprocessing_modelling:38 - nan
2023-08-27 15:58:

Total processing time: 5.78125
(0, 10)
No deliveries from 2022-10-29 to 2022-10-30
(0, 10)
No deliveries from 2022-10-30 to 2022-10-31


### Monday, Wednesday, Friday

In [22]:
get_results(df_dc, list_bi_daily, "bidaily", 1, truck_capacity = truck_capacity, mode = mode, algorithm = algorithm)

2023-08-20 12:18:54.613 | INFO     | __main__:get_results:6 - From 2022-01-03 to 2022-01-05
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-20 12:18:54.626 | INFO     | src.co2_modells:preprocessing_modelling:38 - 140.736
2023-08-20 12:18:54.646 | INFO     | src.co2_modells:co2_modell:71 - 0.4946923076923077


(203, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(25, 12)
(66, 16)


2023-08-20 12:19:06.002 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.007 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.012 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.015 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.018 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.023 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.026 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.046 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:19:06.067 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-20 12:19:06.128 | INFO     | __main__:get_results:6 - From 2022-01-05 to 2022-01-07
C:\Users\ma

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(31, 12)
(58, 16)


2023-08-20 12:47:40.959 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.959 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.959 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.975 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.975 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.975 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.986 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.990 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:40.990 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)


Total processing time: 8.828125


,date from,date to,co2 road,distance road,routes road,processing time road,co2 railroad,distance railroad,terminal allocation,routes railroad,processing time railroad,volume
0,2022-01-03 08:18:00,2022-01-04 16:14:00,9026.021415,10368837,"[[DC2, C472, C149, C777, C190, C344, C336, C81...",0.531250,11586.110346,17476293.0,"[[C163, C787, C676, C124, C697, C293, C778, C1...","{'T1': [['T1', 'C284', 'C238', 'C55', 'C338', ...",9.796875,1.0
1,2022-01-05 10:01:00,2022-01-06 16:31:00,9588.812566,11005829,"[[DC2, C385, C415, C448, C660, C1220, C149, C7...",0.734375,11524.931955,17381513.0,"[[C341, C676, C949, C697, C293, C353, C113, C8...","{'T1': [['T1', 'C284', 'C196', 'C338', 'C273',...",9.015625,1.0
2,2022-01-07 08:58:00,2022-01-07 17:40:00,6623.593559,7865070,"[[DC2, C269, C190, C907, C1360, C448, C898, C4...",0.984375,9409.364300,14895699.0,"[[C697, C293, C661, C273, C883, C55, C196, C73...","{'T1': [['T1', 'C55', 'C196', 'C338', 'C883', ...",9.578125,1.0
3,2022-01-10 08:44:00,2022-01-11 16:40:00,9203.288736,10552539,"[[DC2, C185, C481, C1488, C269, C152, C190, C1...",0.859375,11395.975718,17252307.0,"[[C1485, C676, C1377, C778, C353, C113, C661, ...","{'T1': [['T1', 'C935', 'C284', 'C55', 'C1419',...",9.875000,1.0
4,2022-01-12 08:52:00,2022-01-13 16:57:00,8680.166099,10097891,"[[DC2, C217, C1220, C907, C405, C448, C660, C3...",1.000000,10861.118337,16617318.0,"[[C1485, C341, C787, C124, C697, C353, C113, C...","{'T1': [['T1', 'C284', 'C238', 'C196', 'C55', ...",9.953125,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,2022-12-19 07:37:00,2022-12-20 15:35:00,9955.901310,11343882,"[[DC2, C240, C1183, C675, C1774, C691, C435, C...",0.500000,11052.572876,16809778.0,"[[C1953, C2307, C341, C697, C778, C113, C463, ...","{'T1': [['T1', 'C284', 'C196', 'C55', 'C661', ...",7.406250,1.0
151,2022-12-21 08:47:00,2022-12-22 17:07:00,9403.748327,10849885,"[[DC2, C3541, C4788, C7178, C435, C502, C19, C...",0.781250,11503.138333,17398543.0,"[[C1961, C1953, C124, C697, C778, C113, C661, ...","{'T1': [['T1', 'C1961', 'C1953', 'C124', 'C697...",8.437500,1.0
152,2022-12-23 08:45:00,2022-12-23 16:52:00,7828.427726,9282959,"[[DC2, C4788, C1360, C405, C435, C502, C19, C5...",0.671875,9631.094626,15141319.0,"[[C697, C353, C113, C661, C694, C273, C284, C7...","{'T1': [['T1', 'C284', 'C338', 'C694', 'C661',...",7.687500,1.0
153,2022-12-26 08:51:00,2022-12-27 16:28:00,9629.305994,11123171,"[[DC2, C472, C4788, C1220, C660, C448, C1360, ...",0.718750,11874.623674,17807116.0,"[[C1953, C163, C341, C787, C850, C949, C113, C...","{'T1': [['T1', 'C1181', 'C850', 'C787', 'C341'...",7.640625,1.0


In [23]:
get_results(df_dc, list_bi_daily, "bidaily", 5, truck_capacity = truck_capacity, mode = mode, algorithm = algorithm)

2023-08-20 12:47:42.112 | INFO     | __main__:get_results:6 - From 2022-01-03 to 2022-01-05
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-20 12:47:42.122 | INFO     | src.co2_modells:preprocessing_modelling:38 - 703.6800000000001
2023-08-20 12:47:42.137 | INFO     | src.co2_modells:co2_modell:71 - 2.5013076923076922


(203, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(25, 12)
(66, 16)


2023-08-20 12:47:53.302 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.302 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.314 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.318 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.318 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.318 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.318 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.336 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 12:47:53.341 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-20 12:47:53.350 | INFO     | __main__:get_results:6 - From 2022-01-05 to 2022-01-07
C:\Users\ma

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(31, 12)
(58, 16)


2023-08-20 13:16:35.089 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.094 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.096 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.103 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.108 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.108 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.115 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.120 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:35.120 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)


Total processing time: 8.390625


,date from,date to,co2 road,distance road,routes road,processing time road,co2 railroad,distance railroad,terminal allocation,routes railroad,processing time railroad,volume
0,2022-01-03 08:18:00,2022-01-04 16:14:00,10772.242684,11843062,"[[DC2, C813, C251, C264, C31, C339, C337, C340...",0.859375,12104.708356,17514367.0,"[[C163, C787, C676, C124, C697, C293, C778, C1...","{'T1': [['T1', 'C284', 'C238', 'C55', 'C338', ...",8.265625,5.0
1,2022-01-05 10:01:00,2022-01-06 16:31:00,11484.459893,12531629,"[[DC2, C437, C813, C922, C16, C282, C656, C915...",0.734375,12098.428680,17418308.0,"[[C341, C676, C949, C697, C293, C353, C113, C8...","{'T1': [['T1', 'C284', 'C196', 'C338', 'C273',...",8.125000,5.0
2,2022-01-07 08:58:00,2022-01-07 17:40:00,7231.484099,8002474,"[[DC2, C302, C517, C759, C573, C201, C1322, C9...",0.734375,9636.877850,14903670.0,"[[C697, C293, C661, C273, C883, C55, C196, C73...","{'T1': [['T1', 'C55', 'C196', 'C338', 'C883', ...",8.437500,5.0
3,2022-01-10 08:44:00,2022-01-11 16:40:00,9944.673641,10772577,"[[DC2, C185, C481, C1488, C1114, C95, C1484, C...",0.812500,11951.717926,17268611.0,"[[C1485, C676, C1377, C778, C353, C113, C661, ...","{'T1': [['T1', 'C935', 'C284', 'C55', 'C1419',...",8.796875,5.0
4,2022-01-12 08:52:00,2022-01-13 16:57:00,10949.310076,12122082,"[[DC2, C584, C77, C410, C369, C365, C1773, C33...",0.843750,11249.705771,16606402.0,"[[C1485, C341, C787, C124, C697, C353, C113, C...","{'T1': [['T1', 'C284', 'C238', 'C196', 'C55', ...",8.437500,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,2022-12-19 07:37:00,2022-12-20 15:35:00,11626.554299,12791857,"[[DC2, C249, C272, C19, C439, C743, C1071, C57...",0.671875,11644.052331,16827961.0,"[[C1953, C2307, C341, C697, C778, C113, C463, ...","{'T1': [['T1', 'C284', 'C196', 'C55', 'C661', ...",7.593750,5.0
151,2022-12-21 08:47:00,2022-12-22 17:07:00,10648.288801,11750254,"[[DC2, C271, C290, C2608, C302, C4034, C480, C...",0.906250,11997.764742,17406790.0,"[[C1961, C1953, C124, C697, C778, C113, C661, ...","{'T1': [['T1', 'C1961', 'C1953', 'C124', 'C697...",7.953125,5.0
152,2022-12-23 08:45:00,2022-12-23 16:52:00,7846.735251,8851180,"[[DC2, C152, C4788, C1360, C405, C435, C2689, ...",0.812500,9953.568419,15141319.0,"[[C697, C353, C113, C661, C694, C273, C284, C7...","{'T1': [['T1', 'C284', 'C338', 'C694', 'C661',...",7.468750,5.0
153,2022-12-26 08:51:00,2022-12-27 16:28:00,11275.975201,12455041,"[[DC2, C385, C320, C813, C339, C340, C737, C91...",0.734375,12378.672114,17799449.0,"[[C1953, C163, C341, C787, C850, C949, C113, C...","{'T1': [['T1', 'C1181', 'C850', 'C787', 'C341'...",8.421875,5.0


In [24]:
get_results(df_dc, list_bi_daily, "bidaily", 10, truck_capacity = truck_capacity, mode = mode, algorithm = algorithm)

2023-08-20 13:16:36.079 | INFO     | __main__:get_results:6 - From 2022-01-03 to 2022-01-05
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-20 13:16:36.094 | INFO     | src.co2_modells:preprocessing_modelling:38 - 1407.3600000000001
2023-08-20 13:16:36.108 | INFO     | src.co2_modells:co2_modell:71 - 5.009


(203, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(25, 12)
(66, 16)


2023-08-20 13:16:47.300 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.301 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.301 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.317 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.322 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.331 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.331 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.331 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:16:47.347 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-20 13:16:47.379 | INFO     | __main__:get_results:6 - From 2022-01-05 to 2022-01-07
C:\Users\ma

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(31, 12)
(58, 16)


2023-08-20 13:45:28.610 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.625 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.625 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.625 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.641 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.641 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.641 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.641 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:28.656 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)


Total processing time: 6.15625


,date from,date to,co2 road,distance road,routes road,processing time road,co2 railroad,distance railroad,terminal allocation,routes railroad,processing time railroad,volume
0,2022-01-03 08:18:00,2022-01-04 16:14:00,14123.772035,15511931,"[[DC2, C419, C258, C13, C225, C609, C67, C83, ...",0.921875,12442.580009,17561091.0,"[[C163, C787, C676, C124, C697, C293, C778, C1...","{'T1': [['T1', 'C284', 'C238', 'C55', 'C338', ...",7.890625,10.0
1,2022-01-05 10:01:00,2022-01-06 16:31:00,12559.707264,13753417,"[[DC2, C160, C100, C161, C32, C902, C479, C62,...",0.781250,12625.921883,17704531.0,"[[C341, C676, C949, C697, C293, C353, C113, C8...","{'T1': [['T1', 'C284', 'C196', 'C338', 'C273',...",6.781250,10.0
2,2022-01-07 08:58:00,2022-01-07 17:40:00,8715.848569,9460226,"[[DC2, C190, C517, C1114, C759, C95, C573, C13...",0.515625,9948.918677,14994678.0,"[[C697, C293, C661, C273, C883, C55, C196, C73...","{'T1': [['T1', 'C55', 'C196', 'C338', 'C883', ...",6.765625,10.0
3,2022-01-10 08:44:00,2022-01-11 16:40:00,12558.255166,13839968,"[[DC2, C385, C240, C200, C675, C251, C264, C97...",0.734375,12700.782832,17813706.0,"[[C1485, C676, C1377, C778, C353, C113, C661, ...","{'T1': [['T1', 'C935', 'C284', 'C55', 'C1419',...",6.671875,10.0
4,2022-01-12 08:52:00,2022-01-13 16:57:00,11152.208699,12316112,"[[DC2, C269, C759, C95, C201, C601, C716, C248...",0.671875,11496.304736,16680770.0,"[[C1485, C341, C787, C124, C697, C353, C113, C...","{'T1': [['T1', 'C284', 'C238', 'C196', 'C55', ...",7.671875,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,2022-12-19 07:37:00,2022-12-20 15:35:00,12993.843727,14241274,"[[DC2, C95, C201, C1415, C1118, C133, C601, C1...",0.734375,11940.919746,16921052.0,"[[C1953, C2307, C341, C697, C778, C113, C463, ...","{'T1': [['T1', 'C284', 'C196', 'C55', 'C661', ...",6.843750,10.0
151,2022-12-21 08:47:00,2022-12-22 17:07:00,12494.991745,13590543,"[[DC2, C474, C385, C320, C813, C514, C23, C214...",0.828125,12214.998053,17508886.0,"[[C1961, C1953, C124, C697, C778, C113, C661, ...","{'T1': [['T1', 'C1961', 'C1953', 'C124', 'C697...",6.968750,10.0
152,2022-12-23 08:45:00,2022-12-23 16:52:00,8501.509128,9228039,"[[DC2, C4788, C1360, C405, C435, C336, C363, C...",0.750000,10141.080295,15266186.0,"[[C697, C353, C113, C661, C694, C273, C284, C7...","{'T1': [['T1', 'C284', 'C338', 'C694', 'C661',...",6.875000,10.0
153,2022-12-26 08:51:00,2022-12-27 16:28:00,12819.781223,14153685,"[[DC2, C437, C660, C336, C1808, C1587, C103, C...",0.578125,12803.436330,17983623.0,"[[C1953, C163, C341, C787, C850, C949, C113, C...","{'T1': [['T1', 'C1181', 'C850', 'C787', 'C341'...",6.031250,10.0


### Weekly

In [25]:
get_results(df_dc, list_mondays, "weekly", 1, truck_capacity = truck_capacity, mode = mode, algorithm = algorithm)

2023-08-20 13:45:29.412 | INFO     | __main__:get_results:6 - From 2022-01-03 to 2022-01-10
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-20 13:45:29.428 | INFO     | src.co2_modells:preprocessing_modelling:38 - 359.74399999999997
2023-08-20 13:45:29.429 | INFO     | src.co2_modells:co2_modell:71 - 1.2986923076923076


(324, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(56, 12)
(123, 16)


2023-08-20 13:45:40.638 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.654 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.654 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.654 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.654 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.669 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.669 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.669 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:45:40.678 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-20 13:45:40.707 | INFO     | __main__:get_results:6 - From 2022-01-10 to 2022-01-17
C:\Users\ma

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(49, 12)
(105, 16)


2023-08-20 13:55:16.273 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.289 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.289 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.299 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.305 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.308 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.318 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.321 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:16.325 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)


Total processing time: 5.640625


,date from,date to,co2 road,distance road,routes road,processing time road,co2 railroad,distance railroad,terminal allocation,routes railroad,processing time railroad,volume
0,2022-01-03 08:18:00,2022-01-07 17:40:00,11571.976387,12955951,"[[DC2, C390, C200, C1183, C675, C243, C250, C9...",0.812500,13692.862189,19708493.0,"[[C163, C341, C787, C676, C949, C124, C697, C2...","{'T1': [['T1', 'C79', 'C886', 'C113', 'C697', ...",6.171875,1.0
1,2022-01-10 08:44:00,2022-01-14 17:16:00,12149.312403,13612477,"[[DC2, C271, C270, C275, C68, C266, C702, C584...",0.640625,13281.928052,19248663.0,"[[C1485, C341, C787, C676, C1377, C850, C124, ...","{'T1': [['T1', 'C935', 'C284', 'C238', 'C55', ...",6.750000,1.0
2,2022-01-17 10:13:00,2022-01-21 16:46:00,12273.202492,13595965,"[[DC2, C223, C156, C778, C300, C16, C282, C656...",0.765625,13983.150984,20058517.0,"[[C1961, C1953, C1265, C1485, C341, C787, C676...","{'T1': [['T1', 'C886', 'C463', 'C113', 'C353',...",7.015625,1.0
3,2022-01-24 08:54:00,2022-01-28 18:11:00,11462.487248,12807569,"[[DC2, C1626, C215, C1488, C1114, C494, C2092,...",0.796875,13376.716918,19324898.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C935', 'C284', 'C238', 'C196',...",7.093750,1.0
4,2022-01-31 08:40:00,2022-02-04 16:48:00,12509.198601,13986309,"[[DC2, C271, C2608, C302, C386, C275, C212, C6...",0.718750,13917.621776,19885312.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C935', 'C284', 'C238', 'C196',...",7.312500,1.0
5,2022-02-07 08:34:00,2022-02-12 11:04:00,11947.967391,13345459,"[[DC2, C532, C272, C249, C277, C41, C52, C439,...",0.562500,13720.506301,19811630.0,"[[C1961, C1953, C1485, C163, C341, C850, C949,...","{'T1': [['T1', 'C79', 'C666', 'C886', 'C463', ...",6.359375,1.0
6,2022-02-14 07:33:00,2022-02-18 16:52:00,12095.357422,13382426,"[[DC2, C269, C1114, C95, C1322, C201, C1415, C...",0.796875,13868.082345,19838761.0,"[[C1961, C1953, C1485, C163, C2307, C341, C676...","{'T1': [['T1', 'C79', 'C886', 'C353', 'C341', ...",6.625000,1.0
7,2022-02-21 09:04:00,2022-02-25 17:22:00,13090.171096,14516485,"[[DC2, C269, C190, C277, C249, C19, C41, C743,...",0.671875,14328.635915,20334881.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C293', 'C850', 'C1377', 'C676'...",7.453125,1.0
8,2022-02-28 08:38:00,2022-03-04 19:30:00,13450.045933,15099087,"[[DC2, C1244, C240, C775, C251, C250, C683, C2...",0.750000,14199.860862,20143164.0,"[[C1961, C1953, C1485, C163, C2307, C341, C371...","{'T1': [['T1', 'C1181', 'C1377', 'C676', 'C850...",6.953125,1.0
9,2022-03-07 08:41:00,2022-03-11 17:14:00,12449.296461,13722957,"[[DC2, C3444, C223, C778, C298, C683, C862, C7...",0.687500,13931.375561,19891950.0,"[[C1961, C1953, C1485, C163, C2307, C341, C787...","{'T1': [['T1', 'C1181', 'C293', 'C353', 'C113'...",6.843750,1.0


In [26]:
get_results(df_dc, list_mondays, "weekly", 5, truck_capacity = truck_capacity, mode = mode, algorithm = algorithm)

2023-08-20 13:55:18.038 | INFO     | __main__:get_results:6 - From 2022-01-03 to 2022-01-10
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-20 13:55:18.047 | INFO     | src.co2_modells:preprocessing_modelling:38 - 1798.72
2023-08-20 13:55:18.067 | INFO     | src.co2_modells:co2_modell:71 - 6.542461538461539


(324, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(56, 12)
(123, 16)


2023-08-20 13:55:29.280 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.280 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.295 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.301 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.301 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.311 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.311 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.311 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 13:55:29.311 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-20 13:55:29.347 | INFO     | __main__:get_results:6 - From 2022-01-10 to 2022-01-17
C:\Users\ma

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(49, 12)
(105, 16)


2023-08-20 14:05:05.875 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.875 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.875 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.891 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.891 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.891 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.907 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.907 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:05.907 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)


Total processing time: 6.984375


,date from,date to,co2 road,distance road,routes road,processing time road,co2 railroad,distance railroad,terminal allocation,routes railroad,processing time railroad,volume
0,2022-01-03 08:18:00,2022-01-07 17:40:00,15620.409616,16912471,"[[DC2, C472, C907, C1360, C435, C160, C161, C1...",0.453125,15436.423385,20808839.0,"[[C163, C341, C787, C676, C949, C124, C697, C2...","{'T1': [['T1', 'C79', 'C886', 'C113', 'C697', ...",6.812500,5.0
1,2022-01-10 08:44:00,2022-01-14 17:16:00,16000.219371,17873290,"[[DC2, C750, C962, C437, C813, C922, C264, C12...",0.828125,14412.079485,19670600.0,"[[C1485, C341, C787, C676, C1377, C850, C124, ...","{'T1': [['T1', 'C935', 'C284', 'C238', 'C55', ...",7.125000,5.0
2,2022-01-17 10:13:00,2022-01-21 16:46:00,16965.769998,18536634,"[[DC2, C88, C18, C87, C946, C947, C331, C790, ...",0.812500,15108.849873,20670757.0,"[[C1961, C1953, C1265, C1485, C341, C787, C676...","{'T1': [['T1', 'C196', 'C55', 'C1419', 'C637',...",7.031250,5.0
3,2022-01-24 08:54:00,2022-01-28 18:11:00,15378.577858,16941347,"[[DC2, C217, C149, C190, C1291, C1114, C1295, ...",0.687500,14582.673683,19737282.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C935', 'C284', 'C238', 'C196',...",7.500000,5.0
4,2022-01-31 08:40:00,2022-02-04 16:48:00,17496.840923,19004312,"[[DC2, C494, C2092, C1068, C122, C432, C101, C...",0.718750,15366.567717,20865912.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C196', 'C338', 'C883', 'C694',...",6.703125,5.0
5,2022-02-07 08:34:00,2022-02-12 11:04:00,16171.018601,17573727,"[[DC2, C532, C19, C446, C549, C1587, C1299, C1...",0.640625,15085.151828,20433005.0,"[[C1961, C1953, C1485, C163, C341, C850, C949,...","{'T1': [['T1', 'C79', 'C666', 'C886', 'C463', ...",7.640625,5.0
6,2022-02-14 07:33:00,2022-02-18 16:52:00,16969.806708,18516156,"[[DC2, C336, C158, C1459, C21, C978, C707, C43...",0.843750,15611.513151,21029760.0,"[[C1961, C1953, C1485, C163, C2307, C341, C676...","{'T1': [['T1', 'C79', 'C886', 'C353', 'C341', ...",7.656250,5.0
7,2022-02-21 09:04:00,2022-02-25 17:22:00,20066.133350,22079086,"[[DC2, C286, C145, C39, C57, C65, C69, C128, C...",0.781250,16370.623560,21776605.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C79', 'C886', 'C463', 'C73', '...",7.312500,5.0
8,2022-02-28 08:38:00,2022-03-04 19:30:00,18077.560146,19698260,"[[DC2, C635, C84, C446, C549, C725, C557, C181...",0.875000,15826.472759,21174007.0,"[[C1961, C1953, C1485, C163, C2307, C341, C371...","{'T1': [['T1', 'C1181', 'C1377', 'C676', 'C850...",6.687500,5.0
9,2022-03-07 08:41:00,2022-03-11 17:14:00,19544.571186,21335621,"[[DC2, C255, C792, C695, C1351, C83, C70, C722...",0.671875,15625.525567,21018770.0,"[[C1961, C1953, C1485, C163, C2307, C341, C787...","{'T1': [['T1', 'C1181', 'C661', 'C694', 'C778'...",5.906250,5.0


In [27]:
get_results(df_dc, list_mondays, "weekly", 10, truck_capacity = truck_capacity, mode = mode, algorithm = algorithm)

2023-08-20 14:05:07.496 | INFO     | __main__:get_results:6 - From 2022-01-03 to 2022-01-10
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sender weight (kg)"] = df["Sender weight (kg)"]*volume
2023-08-20 14:05:07.512 | INFO     | src.co2_modells:preprocessing_modelling:38 - 3597.44
2023-08-20 14:05:07.528 | INFO     | src.co2_modells:co2_modell:71 - 13.098


(324, 7)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(56, 12)
(123, 16)


2023-08-20 14:05:18.756 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.756 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.767 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.767 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.767 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.782 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.782 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.782 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:05:18.782 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)
2023-08-20 14:05:18.822 | INFO     | __main__:get_results:6 - From 2022-01-10 to 2022-01-17
C:\Users\ma

...

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  distances['Dmin'] = distances.min(axis=1)


(49, 12)
(105, 16)


2023-08-20 14:14:55.798 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.814 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.817 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.821 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.824 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.829 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.832 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.832 | INFO     | src.co2:mesoscopic_train:136 - 1
2023-08-20 14:14:55.841 | INFO     | src.co2:mesoscopic_train:136 - 1
C:\Users\marle\AppData\Local\Temp\ipykernel_20148\1747787820.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(dict_results, ignore_index=True)


Total processing time: 8.515625


,date from,date to,co2 road,distance road,routes road,processing time road,co2 railroad,distance railroad,terminal allocation,routes railroad,processing time railroad,volume
0,2022-01-03 08:18:00,2022-01-07 17:40:00,22190.411217,24479643,"[[DC2, C591, C577, C323, C574, C76, C213, C513...",0.687500,16288.265268,21628401.0,"[[C163, C341, C787, C676, C949, C124, C697, C2...","{'T1': [['T1', 'C293', 'C676', 'C124', 'C787',...",6.578125,10.0
1,2022-01-10 08:44:00,2022-01-14 17:16:00,20665.517603,22694606,"[[DC2, C472, C660, C136, C810, C722, C36, C119...",0.625000,16061.195023,21366494.0,"[[C1485, C341, C787, C676, C1377, C850, C124, ...","{'T1': [['T1', 'C935', 'C55', 'C196', 'C338', ...",3.921875,10.0
2,2022-01-17 10:13:00,2022-01-21 16:46:00,26382.777629,28696161,"[[DC2, C34, C27, C554, C80, C4, C321, C1682, C...",0.687500,16953.344248,22102986.0,"[[C1961, C1953, C1265, C1485, C341, C787, C676...","{'T1': [['T1', 'C886', 'C463', 'C565', 'C73', ...",7.234375,10.0
3,2022-01-24 08:54:00,2022-01-28 18:11:00,22042.791912,24101552,"[[DC2, C472, C660, C435, C691, C2104, C13, C22...",0.437500,17353.325919,22665335.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C935', 'C284', 'C238', 'T1'], ...",5.234375,10.0
4,2022-01-31 08:40:00,2022-02-04 16:48:00,24539.979902,26716853,"[[DC2, C588, C1363, C72, C1057, C98, C1009, C3...",0.515625,17747.085538,23008924.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C935', 'C284', 'C883', 'C694',...",6.796875,10.0
5,2022-02-07 08:34:00,2022-02-12 11:04:00,22824.620640,24971295,"[[DC2, C149, C336, C590, C535, C431, C1459, C2...",0.890625,16096.713120,21257659.0,"[[C1961, C1953, C1485, C163, C341, C850, C949,...","{'T1': [['T1', 'C1025', 'C1379', 'C565', 'C73'...",6.500000,10.0
6,2022-02-14 07:33:00,2022-02-18 16:52:00,24710.223644,27041786,"[[DC2, C709, C136, C51, C762, C84, C89, C469, ...",0.625000,17243.017692,22465415.0,"[[C1961, C1953, C1485, C163, C2307, C341, C676...","{'T1': [['T1', 'C850', 'C1377', 'C676', 'C124'...",7.296875,10.0
7,2022-02-21 09:04:00,2022-02-25 17:22:00,27477.451412,30159939,"[[DC2, C72, C24, C1775, C37, C742, C242, C224,...",0.796875,18456.260139,23747520.0,"[[C1961, C1953, C1265, C1485, C163, C2307, C34...","{'T1': [['T1', 'C284', 'C238', 'C1025', 'C73',...",7.109375,10.0
8,2022-02-28 08:38:00,2022-03-04 19:30:00,25384.826603,27831290,"[[DC2, C635, C2149, C102, C761, C15, C165, C16...",0.625000,17226.069214,22436420.0,"[[C1961, C1953, C1485, C163, C2307, C341, C371...","{'T1': [['T1', 'C850', 'C1377', 'C676', 'C124'...",6.578125,10.0
9,2022-03-07 08:41:00,2022-03-11 17:14:00,28128.530859,30586255,"[[DC2, C437, C502, C249, C19, C1502, DC2], [DC...",0.718750,18580.450243,23813092.0,"[[C1961, C1953, C1485, C163, C2307, C341, C787...","{'T1': [['T1', 'C284', 'C196', 'C55', 'T1'], [...",7.343750,10.0


***

In [ ]:
df_results_total = pd.concat([
    create_table_total(df_results_daily, "daily planning"),
    create_table_total(df_results_bi_daily, "bi-daily planning"),
    create_table_total(df_results_weekly, "weekly planning")], axis = 1)
df_results_total

In [ ]:
df_results_weekly.head(5)[["co2 road", "co2 railroad"]].T

In [ ]:
writer = pd.ExcelWriter('results/results_base.xlsx')
df_results_daily_mean.to_excel(writer,'Daily average')
df_results_total.to_excel(writer,'Total')
writer.save()
writer.close()

***

## Scenario 3: Combining two DC's

### Daily

In [ ]:
from src.evaluation_multi import co2_modell_multi

In [ ]:
df_combined = pd.concat([df_dc1, df_dc2]).head(1000)

In [ ]:
columns = ["date from",
            "date to",
            "co2", 
            "distance", 
            "processing time",
            "df shape"]
df_results_combined = pd.DataFrame(columns = columns)
date_from = "2022-01-03"
date_to = "2022-01-04"
dcs = ["DC1", "DC2"]
dict_results, df_results_details_combined = co2_modell_multi(dcs, df_combined, df_distance_matrix, dict_terminals, date_from, date_to)
# df_results_combined = df_results_combined.append(pd.DataFrame(dict_results, index = ["combined"]))
# for dc in dcs:
#     dict_results, df_results_details = co2_modell_multi([dc], df_combined, df_distance_matrix, dict_terminals, date_from, date_to)
#     df_results_combined = df_results_combined.append(pd.DataFrame(dict_results, index = [f"{dc} only"]))

In [ ]:
df_results_details_combined